## Initial

In [ ]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENROUTER_API_KEY = os.getenv("SHISHA_API_KEY")
GROQCLOUD_API_KEY = os.getenv("GROQCLOUD_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
import requests
from openai import OpenAI 

OPENROUTER_CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

OPENAI_CLIENT = OpenAI(api_key=OPENAI_API_KEY)

def call_openai_api(prompt: str) -> str:
    try:
        response = OPENAI_CLIENT.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        )

        reply = response.choices[0].message.content

        return reply 
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None

def call_openrouter_api(prompt: str) -> str:
    try:
        completion = OPENROUTER_CLIENT.chat.completions.create(
            model="moonshotai/kimi-dev-72b:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None
    
def call_groq_api(prompt: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQCLOUD_API_KEY}"
    }
    payload = {
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        return result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"GROQ API call failed: {e}")
        return None

In [ ]:
AUGMENTATION_PROMPT = """Paraphase with the same meaning & language as this question:
- 3 command-style questions with an incomplete sentence structure (no need for full subject-verb), but the hotel name must be included.
- 2 short, concise questions.
Output: a list of 5 string (no explanation, no extra text, remove ordinal number).
The question: {question}
"""

## Top-3 questions

In [ ]:
import json

file_path = "./test.json"
list_questions = []

with open(file_path, 'r', encoding='utf-8') as file:
    raw_data = json.load(file)

for data in raw_data:
    list_questions.append(data['conversations'][0]['human'])

for question in list_questions[:3]:
    print(question)

## Send each question to LLM for variation

In [ ]:
list_conversations = []

for index, question in enumerate(list_questions[:4]):
    prompt = AUGMENTATION_PROMPT.format(question=question)
    response = call_openai_api(prompt)

    print(response)
    print("\n===\n")


In [ ]:
print(raw_data[0])
print(raw_data[1])
print(raw_data[2])

In [ ]:
questions = ["1", "2", "3"]
gpt_answers = raw_data[0]['conversations'][0]['gpt']
new_conversations = []

for question in questions:
    temp_conversation = {
        'conversations': [
            {'human': question, 'gpt': gpt_answers}
        ]
    }

    new_conversations.append(temp_conversation)

new_conversations